In [1]:
#!pip install tensorflow opencv-python mediapipe sklearn matplotlib

In [2]:
import cv2
import numpy as np
import os
import time
import mediapipe as mp
from tensorflow import keras
from scipy import stats

In [3]:
model = keras.models.load_model('symptom.h5')

2022-12-14 12:38:21.457528: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-14 12:38:21.457945: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-14 12:38:21.460166: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [4]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 21)                693       
Total params: 597,269
Trainable params: 597,269
Non-trainable params: 0
__________________________________________________

In [9]:
import os

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

actions = os.listdir(DATA_PATH)

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [10]:
actions = np.array(actions)

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]
                break
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('Webcam Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

print(sentence)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
QObject::moveToThread: Current thread (0x55cadb478b50) is not the object's thread (0x55cadb3f55a0).
Cannot move to target thread (0x55cadb478b50)

QObject::moveToThread: Current thread (0x55cadb478b50) is not the object's thread (0x55cadb3f55a0).
Cannot move to target thread (0x55cadb478b50)

QObject::moveToThread: Current thread (0x55cadb478b50) is not the object's thread (0x55cadb3f55a0).
Cannot move to target thread (0x55cadb478b50)

QObject::moveToThread: Current thread (0x55cadb478b50) is not the object's thread (0x55cadb3f55a0).
Cannot move to target thread (0x55cadb478b50)

QObject::moveToThread: Current thread (0x55cadb478b50) is not the object's thread (0x55cadb3f55a0).
Cannot move to target thread (0x55cadb478b50)

QObject::moveToThread: Current thread (0x55cadb478b50) is not the object's thread (0x55cadb3f55a0).
Cannot move to target thread (0x55cadb478b50)

QObject::moveToThread: Current thread (0x55cadb478b50) is not 

2022-12-14 12:38:29.504626: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-12-14 12:38:29.505207: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999965000 Hz


continuous_sneezing
continuous_sneezing
continuous_sneezing
continuous_sneezing
diarrhoea
diarrhoea
diarrhoea
diarrhoea
diarrhoea
diarrhoea
diarrhoea
diarrhoea
diarrhoea
diarrhoea
continuous_sneezing
continuous_sneezing
continuous_sneezing
continuous_sneezing
continuous_sneezing
continuous_sneezing
continuous_sneezing
continuous_sneezing
continuous_sneezing
continuous_sneezing
continuous_sneezing
continuous_sneezing
continuous_sneezing
continuous_sneezing
continuous_sneezing
continuous_sneezing
runny_nose
continuous_sneezing
continuous_sneezing
continuous_sneezing
runny_nose
continuous_sneezing
runny_nose
continuous_sneezing
runny_nose
continuous_sneezing
runny_nose
continuous_sneezing
runny_nose
continuous_sneezing
runny_nose
continuous_sneezing
runny_nose
runny_nose
runny_nose
runny_nose
runny_nose
runny_nose
runny_nose
runny_nose
continuous_sneezing
runny_nose
runny_nose
runny_nose
runny_nose
runny_nose
runny_nose
runny_nose
runny_nose
runny_nose
continuous_sneezing
runny_nose
conti

In [14]:
import pickle

In [15]:
with open('disease_predict_model.pkl','rb') as modelFile:
     model = pickle.load(modelFile)

In [16]:
disease_test = np.zeros(21, dtype=int)
disease_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [17]:
import json

In [18]:
with open('prognosis.json', 'r') as openfile:
 
    # Reading from json file
    dis_list = json.load(openfile)

In [19]:
print(dis_list)

{'0': 'Allergy', '1': 'Bronchial Asthma', '2': 'Chicken pox', '3': 'Common Cold', '4': 'Covid', '5': 'Dengue', '6': 'Diabetes', '7': 'Drug Reaction', '8': 'Fungal infection', '9': 'Gastroenteritis', '10': 'Heart attack', '11': 'Hypertension', '12': 'Hyperthyroidism', '13': 'Hypothyroidism', '14': 'Jaundice', '15': 'Malaria', '16': 'Migraine', '17': 'Peptic ulcer disease', '18': 'Pneumonia', '19': 'Psoriasis', '20': 'Tuberculosis', '21': 'Typhoid'}


In [20]:
with open('column_names.json', 'r') as openfile:
 
    # Reading from json file
    col_list = json.load(openfile)

In [21]:
saved_index = []
for sen in sentence:
    for key, value in col_list.items():
        if value == sen:
            saved_index.append(int(key))
print(saved_index)

[13, 7, 3, 5, 9]


In [22]:

for i in saved_index:
    disease_test[i] = 1
    
print(disease_test)



[0 0 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0]


In [23]:
p=model.predict([disease_test])

/home/hackerpro/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [24]:
p

array([15])

In [25]:
for key,value in dis_list.items():
    if int(key) == p[0]:
        print(value)
        break

Malaria


In [26]:
cap.release()
cv2.destroyAllWindows()